# 模型推理 - 使用 QLoRA 微调后的 ChatGLM-6B

In [1]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

# 模型ID或本地路径
model_name_or_path = 'THUDM/chatglm3-6b'

In [2]:
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])

# 加载量化后模型(与微调的 revision 保持一致）
base_model = AutoModel.from_pretrained(model_name_or_path,
                                      quantization_config=q_config,
                                      device_map='auto',
                                      trust_remote_code=True,
                                      revision='b098244')

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [3]:
base_model.requires_grad_(False)
base_model.eval()

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,
                                          trust_remote_code=True,
                                          revision='b098244')

## 使用原始 ChatGLM3-6B 模型

In [5]:
input_text = "解释下乾卦是什么？"

In [6]:
response, history = base_model.chat(tokenizer, query=input_text)

In [7]:
print(response)

乾卦是八卦之一，也是八宫图说、易经、易学中的重要元素。乾卦是由两个阴爻夹一个阳爻构成，象征着天、阳气、强盛、积极、刚健等含义。乾卦的卦辞是“元、亨、利、贞”，表示这个卦具有伟大的的开始、通晓、顺利和正定等特性。

乾卦的六爻可以分为阳爻和阴爻，每一卦有六个爻，其中阳爻用“乾”表示，阴爻用“坤”表示。通过排列组合，乾卦可以组成不同的六爻组合，如乾震、乾巽、乾离、乾兑、乾卦、坤卦等。这些组合可以用来分析事物的发展变化和趋势，指导人们做出正确的决策。

乾卦的含义和特性在古代中国哲学和道教中有着重要的地位，它与八卦、易经、易学等密切相关，被广泛应用于医学、农业、战争、政治、建筑等领域。


#### 询问一个64卦相关问题（应该不在 ChatGLM3-6B 预训练数据中）

In [8]:
response, history = base_model.chat(tokenizer, query="周易中的讼卦是什么？", history=history)
print(response)

讼卦是八卦之一，也是八宫图说、易经、易学中的重要元素。讼卦是由两个阴爻夹一个阳爻构成，象征着天、阳气、强盛、积极、刚健等含义。

在周易中，讼卦的卦辞是“天无言，命之不可易；易之不可易，言不可易”。这句话表达了天地自然有其自身的规律和秩序，人类无法改变，只能顺应和适应。

讼卦的意义在于提醒人们要尊重自然、顺应规律，不要过于强求改变事物。在实际生活中，讼卦也可以用来分析事物的发展变化和趋势，指导人们做出正确的决策。

需要注意的是，周易中的八卦是一种哲学符号，它的含义和特性在古代中国哲学和道教中有着重要的地位，被广泛应用于医学、农业、战争、政治、建筑等领域。因此，在理解和学习周易时，需要结合实际情况和领域来综合分析。


## 使用微调后的 ChatGLM3-6B

### 加载 QLoRA Adapter(Epoch=3, automade-dataset(fixed)) - 请根据训练时间戳修改 timestamp 

In [9]:
from peft import PeftModel, PeftConfig

epochs = 3
# timestamp = "20240118_164514"
# timestamp = "20240225_222843"
timestamp = "20240407_001927"

peft_model_path = f"models/{model_name_or_path}-epoch{epochs}-{timestamp}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag=f"ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-{timestamp}"

In [4]:
def compare_chatglm_results(query, base_model, qlora_model, training_tag):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = qlora_model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\n微调后（{training_tag}）：\n{ft_response}")
    return base_response, ft_response

### 微调前后效果对比

In [11]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model, training_tag)

问题：解释下乾卦是什么？

原始输出：
乾卦是周易中的一卦，由两个乾卦叠加而成，代表天象中的乾卦，也就是干燥、坚实、刚健和方形。在周易中，乾卦代表着宇宙的运行和天地间的秩序，同时也代表着君子应当坚贞不屈，以发挥 attendant yoga 的作用。乾卦的核心哲学是：健健不息，积极进取，刚健不挠，坚持正道，完善自我。

乾卦的卦象是由两个乾卦叠加而成，象征着天象中乾卦的形态。在周易中，乾卦代表刚健、坚实、坚定和正派，同时也象征着宇宙的运行和天地间的秩序。乾卦的核心哲学是：健健不息，积极进取，刚健不挠，坚持正道，完善自我。

乾卦的卦象预示着乾卦的出现，预示着宇宙中乾卦的运行和天地间的秩序。乾卦预示着君子应当坚贞不屈，坚持正道，积极进取，刚健不挠，完成 attendant yoga 的使命。在事业、经商、求名、婚恋等方面的运势都有良好的预测。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20240407_001927）：
[gMASK]sop 解释下乾卦是什么？ 在周易中，乾卦是六十四卦之首，由六个阳爻组成，象征着天。它所代表的是刚健、健行、刚健不屈的意境。乾卦的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。

乾卦象征天，为大通而至正。得此卦者，名利双收，应把握机会，争取成果。然而，切勿过于骄傲自满，而应保持谦逊、冷静和警惕。在事业、经商、求名等方面，乾卦皆暗示着大吉大利，但也警示着必须坚持正道、修养德行，方能永远亨通。

在婚恋方面，乾卦提示着阳盛阴衰，但也强调刚柔相济，相互补足，形成美满的结果。在决策方面，则是强调刚健、正直、公允，自强不息的实质，需要修养德行、坚定信念，方能克服困难，消除灾难。


In [12]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model, training_tag)

问题：周易中的讼卦是什么

原始输出：
在周易中，讼卦是一个充满警示的卦象。它由上卦乾（天）和下卦坎（水）组成，代表着天与水背道而驰，形成争讼的局面。虽然事情开始时有利可图，但必须警惕戒惧，因为中间虽然吉利，但最终会带来凶险。对于涉及大川，涉水渡河的行动不利。因此，君子观此卦象，应当慎之又慎，杜绝争讼之事，并在谋事之初谨慎行事。讼卦的核心哲学是要避免争讼，退而让人，求得化解，安于正理，方可避免意外之灾。在事业上，务必避免介入诉讼纠纷的争执之中，与其这样，不如退而让人。即使最终获胜，也难免得失不均。经商方面，要坚持公正、公平、互利的原则，避免冲突，这样会有好结果。而对于求名、婚恋和决策，也都需要慎重行事，避免盲目追求，退让让人，可助事业、婚姻和决策的发展。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20240407_001927）：
[gMASK]sop 周易中的讼卦是什么样子? 在周易中，讼卦是一个充满警示的卦象。它由上卦乾（天）和下卦坎（水）组成，代表着天与水背道而驰，形成争讼的局面。虽然事情开始时有利可图，但必须警惕戒惧，因为中间虽然吉利，但最终会带来凶险。对于涉及大川，涉水渡河的行动不利。因此，君子观此卦象，应当慎之又慎，杜绝争讼之事，并在谋事之初谨慎行事。讼卦的核心哲学是要避免争讼，退而让人，求得化解，安于正理，方可避免意外之灾。在事业上，务必避免介入诉讼纠纷的争执之中，与其这样，不如退而让人。即使最终获胜，也难免得失不均。经商方面，要坚持公正、公平、互利的原则，避免冲突，这样会有好结果。而对于求名、婚恋和决策，也都需要慎重行事，避免盲目追求，退让让人，可助事业、婚姻和决策的发展。


In [13]:
base_response, ft_response = compare_chatglm_results("师卦是什么？", base_model, qlora_model, training_tag)

问题：师卦是什么？

原始输出：
占卜结果 师卦是一个由坎卦（水）和坤卦（地）相叠而成的异卦。这一卦象代表着军队的力量和军情的总指挥，预示着吉祥无灾。象辞中描述了地中有水的情景，寓意着君子应当像大地一样容纳和畜养大众。师卦的解释强调选择德高望重的长者来统率军队，才能获得吉祥无咎。另外，师卦也象征着困难重重，需要包容别人、艰苦努力，及时行事，严于律已。在事业、经商、求名、婚恋等方面的决策中，都需要警惕潜在敌人，小心谨慎，合作与决断兼顾，方能成功。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20240407_001927）：
[gMASK]sop 师卦是什么？ 在周易中，师卦是一个由坎卦（水）和坤卦（地）相叠而成的异卦。这一卦象代表着军队的力量和军情的总指挥，预示着吉祥无灾。象辞中描述了地中有水的情景，寓意着君子应当像大地一样容纳和畜养大众。师卦的解释强调选择德高望重的长者来统率军队，才能获得吉祥无咎。另外，师卦也象征着困难重重，需要包容别人、艰苦努力，及时行事，严于律已。在事业、经商、求名、婚恋等方面的决策中，都需要警惕潜在敌人，小心谨慎，合作与决断兼顾，方能成功。


## 其他模型（错误数据或训练参数）

#### 加载 QLoRA Adapter(Epoch=3, automade-dataset)

In [5]:
from peft import PeftModel, PeftConfig

epochs = 3
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model_e3 = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag = f"ChatGLM3-6B(Epoch=3, automade-dataset)"

In [10]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model_e3, training_tag)

问题：解释下乾卦是什么？

原始输出：
乾卦是周易中的一卦，代表天，象征刚健强劲的特性。它由两个阳爻（代表天）叠加而成，具有刚强的力量和统治天下的能力。在卜卦中，乾卦预示着先考试成绩的失败，但最终会得到补救。在事业和生活中，刚健强力的特性会带来成功。同时，还需要注意天地之变化，以便刚强不会持久，避免遭受失败。

在《象辞》中，乾卦被描述为刚强强劲的君主，具有天下共主之象。在《哲学》中，乾卦被解释为健行不息，刚健有力的象征，同时也预示着刚柔相济的变化。

在解卦中，乾卦表示刚强强力的性格，经商则利，宜冒险，但需谨慎，在事业和生活中坚持刚健强力的策略，刚柔相济，则可成功。

在 parable of the horse, 乾卦的质地被解释为刚强强劲，而经商则利，利为义。在事业和生活中，需要坚持刚健强力的策略，刚柔相济，则可成功。

在解卦中，强调坚持刚强的性格会带来失败，而柔和的性格则能带来成功。因此，在事业和生活中，需要坚持刚健强力的策略，同时注意刚柔相济，则可成功。

在解卦中，提醒人们要刚强强力的性格可能会带来失败，而柔和的性格则能带来成功。因此，在事业和生活中，需要坚持刚健强力的策略，同时注意刚柔相济，则可成功。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset)）：
[gMASK]sop 解释下乾卦是什么？ 乾卦是一个由六条直线组成的卦象，象征着天。它由两个阳爻（代表天）叠加而成，它们相互配合，相互支持，象征着天地的运行和相互依存。在卜战中，乾卦预示着吉祥、勇敢和威武，表示 allies（伙伴）会像天一样运行，并带来利好消息。在决策中，乾卦鼓励人们积极进取，利及他人，表示勇敢者能克服困难，带来吉祥。在选择合作伙伴时，乾卦也预示着会遇到勇敢、刚健的人。



乾卦的哲学含义：



乾卦所代表的是天，是运行和变化的象征，代表着一种刚健、勇敢、进取的态度。在决策中，乾卦鼓励人们积极进取，利及他人，表示勇敢者能克服困难，带来吉祥。在选择合作伙伴时，乾卦也预示着会遇到勇敢、刚健的人。

在解卦时，乾卦表示的是天行健，君子以自强不息。在卜卦时，鼓励人们积极进取，勇敢面对困难，实现自己的目标。

在分析乾卦时，专家们认为，乾卦是一个充满力量和动力的卦象，代表着一种刚健、勇敢、进取的态度。在决策中，乾卦鼓励人们积极进取，利及他人，表示勇

In [11]:
base_response, ft_response = compare_chatglm_results("地水师卦是什么？", base_model, qlora_model_e3, training_tag)

问题：地水师卦是什么？

原始输出：
地水师卦是周易中的一卦，由卦象可知，上卦为坤，代表地，代表顺，为地，为顺为，地下有水，为润，故名为地水师卦。卦名为师，意味着师从师众，克敌益强。

地水师卦的核心哲学是：

* 初，为地实压水，为师出师众。
* 二，为地润湿，为师出师众。
* 三，为地和平，为师渡江。
* 四，为地克水，为师渡江。
* 五，为地和平，为师渡江。
* 六，为地克水，为师渡江。
* 七，为地润湿，为师出师众。
* 八，为地实压水，为师出师众。
* 九，为地和平，为师渡江。
* 十，为地克水，为师渡江。

 summary:

* 初，为地实压水，为师出师众。
* 二，为地润湿，为师出师众。
* 三，为地和平，为师渡江。
* 四，为地克水，为师渡江。
* 五，为地和平，为师渡江。
* 六，为地克水，为师渡江。
* 七，为地润湿，为师出师众。
* 八，为地实压水，为师出师众。
* 九，为地和平，为师渡江。
* 十，为地克水，为师渡江。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset)）：
[gMASK]sop 地水师卦是什么？ 

 地水师卦是周易卦象中的一种，由两个地卦和一个小水卦组成，代表地上的水归流到地下来FM。在卜问时，这个卦象代表着安定、静谧、安顺、平安、吉祥、富厚、安宅等，预示着吉利的事情。


In [12]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model_e3, training_tag)

问题：周易中的讼卦是什么

原始输出：
讼卦是周易中的一卦，由上卦坤（地）和下卦乾（天）组成，代表着天地的 Split卦。讼卦的卦辞描述了天地的不同，以及由此导致的矛盾和争斗。讼卦的核心观念是：天与地不同，相互对待对方的看法，双方都欲向对方证明自己的正确。讼卦提示我们，当遇到矛盾和争斗时，应该保持冷静，双方都需克制自己的情绪，避免矛盾恶化。

讼卦的哲学含义是：矛盾和争斗是不可避免的，但可以通过理智和克制来化解。双方都需要保持冷静，倾听对方的意见，并避免片面主观。通过沟通和和解，双方可以达成共识，解决问题。

讼卦的启示：当遇到矛盾和争斗时，应该保持冷静，倾听对方意见，避免片面主观。通过沟通和解，可以解决问题。

讼卦的时运：讼卦在古时代表诉讼事务，解决纠纷，但往往 predictions were not accurate. Now, the court is not the only way to settle disputes, and people are encouraged to resolve conflicts through negotiation and dialogue.

讼卦的寓言：讼卦的卦辞描述了天地的不同，双方都欲向对方证明自己的正确。这就像是一场诉讼，双方都希望赢得判决。因此，讼卦告诫我们，在解决冲突时，双方都需要保持冷静，互相尊重，并寻求和平解决的方式。

讼卦的详解：讼卦表示天地之间的不同，双方相互对待对方的看法，都希望证明自己的正确。讼卦双方都想要达到自己的目的，互不相让，因此矛盾加剧。讼卦提醒我们，在处理矛盾和争斗时，双方都需要保持冷静，避免情绪化的行为，以理智和克制来解决问题。

讼卦的占卜：在讼卦中，指数为2，表示双方都试图证明自己的正确，但最终需要寻求和解。

讼卦的预测：讼卦往往预示着双方都会坚持自己的立场，互不相让，最终可能导致双方都失败。

讼卦的解卦：决iger（泽），表示双方都需要寻求和解，以避免更大的损失。解卦提醒我们，在解决冲突时，需要寻求和平解决的方式，避免双方都受损。

讼卦的颂词：

乾：刚健 刚毅
坤：柔顺 宽厚

天地的不同，相互对待对方的看法，双方都欲向对方证明自己的正确

利于诉讼，不利 conclusion

讼卦，讼卦，利见大人

和家长争斗，可能得出合理的解决

来源：https://e

#### 加载 QLoRA Adapter(Epoch=50, Overfit, handmade-dataset)

In [5]:
from peft import PeftModel, PeftConfig

epochs = 50
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model_e50_handmade = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag = f"ChatGLM3-6B(Epoch=50, handmade-dataset)"

In [8]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model_e50_handmade, training_tag)

问题：解释下乾卦是什么？

原始输出：
乾卦是周易中的一卦，代表天，由六个阳爻组成，象征着刚健强劲的特性。它由上卦乾（天）和下卦乾（天）组成，上卦为天，下卦为天，因此 named after the pattern. 

在周易中，乾卦预示着天象的刚健强劲，以及人们对待事物时应具有刚健强劲的特性。同时，乾卦也象征着天行健，以及君子刚健健忘的特性。

在卜卦中，乾卦表示天行健，则君子当刚健健忘，以果敢坚毅的态度处理事物。在决策时，应该采取果敢、刚健坚毅的态度，并且积极进取。在决策时应该选择的因素是刚健坚毅。

乾卦的哲学含义是：天行健，君子当刚健健忘，以果敢坚毅的态度处理事物。


微调后（ChatGLM3-6B(Epoch=50, handmade-dataset)）：
[gMASK]sop 解释下乾卦是什么？ 乾卦是周易中的一卦，代表天，由六个阳爻组成，象征着刚健强劲的特性。它象征着刚健强劲的特性，以及天行健的象义。乾卦的卦象为：乾：天 健：健 杀：杀 纳：纳 顺：顺 余：余 利：利 敬：敬 善：善 乾卦的卦辞为：天行健，志意健，刚健强劲的特性，游走在天象之中。

天行健，志意健，刚健强劲的特性，游走在天象之中。

刚健强劲的特性，应当刚强坚毅，志气高昂。

健，游走在天象之中，

天行健，志意健，刚健强劲的特性，游走在天象之中。

刚健强劲的特性，应当刚强坚毅，志气高昂。

天行健，志意健，刚健强劲的特性，游走在天象之中。

刚健强劲的特性，应当刚强坚毅，志气高昂。

乾卦的运势为：天行健，志意健，刚健强劲的特性，游走在天象之中。

天行健，志意健，刚健强劲的特性，游走在天象之中。

刚健强劲的特性，应当刚强坚毅，志气高昂。

天行健，志意健，刚健强劲的特性，游走在天象之中。

刚健强劲的特性，应当刚强坚毅，志气高昂。

乾卦的事业运势为：天行健，志意健，刚健强劲的特性，游走在天象之中。

天行健，志意健，刚健强劲的特性，游走在天象之中。

刚健强劲的特性，应当刚强坚毅，志气高昂。

天行健，志意健，刚健强劲的特性，游走在天象之中。

刚健强劲的特性，应当刚强坚毅，志气高昂。

乾卦的运势解释为：刚强强劲的特性，应当刚强坚毅，志气高昂。

天行健，志意健，刚健强劲的特性，游走在天象之中。

刚健强劲的特性，


In [9]:
base_response, ft_response = compare_chatglm_results("地水师卦", base_model, qlora_model_e50_handmade, training_tag)

问题：地水师卦

原始输出：
地水师卦是周易卦象中的一枚卦，由两个卦象组成，上卦为地，下卦为水。在卜筮时，地水卦表示上卦为阴，下卦为阳，象征地中有水，代表地面上流水的卦象。这个卦象具有宽厚宽厚的好处，预示着吉祥和幸福。

在心理学领域，地水师卦的卦象被解释为宽厚宽厚的好处，代表着宽厚、宽泛之象，预示着吉祥和幸福。在商业领域，这个卦象预示着商业活动顺利，收入增多，利润丰厚。在人际关系方面，这个卦象表示人际关系和谐，互相帮助，相互宽容。

地水师卦的卦象具有宽厚宽厚的好处，代表着宽厚、宽泛之象，预示着吉祥和幸福。在商业领域，这个卦象预示着商业活动顺利，收入增多，利润丰厚。在人际关系方面，这个卦象表示人际关系和谐，互相帮助，相互宽容。


微调后（ChatGLM3-6B(Epoch=50, handmade-dataset)）：
[gMASK]sop 地水师卦


地水师卦是周易卦象中的一枚卦，由两个卦象组合而成，代表一种以水为特征的卦象。在卜卦时，预示着上下一心协作，共同谋求改变现状的信念。

卦象为：上卦为乾（天），下卦为坎（水），两者相背而行，天与水违，因此形成Calamity（灾祸）

。

 Calamity is a warning of the dangers of unity and the need for Caution ( Calamity). It reminds us to trust in the power of水, which has the ability to flow and overcome obstacles, and to be mindful of the potential for harm, and to take precautions to avoid it. It also reminds us to trust in the power of unity, and to work together to overcome difficulties and challenges.

地水师卦的卜卦解释：

上卦为乾（天），下卦为坎（水），两者相背而行，天与水违，因此形成Calamity（灾祸）

。Calamity is a warning of the dangers of unity and the need f

In [10]:
base_response, ft_response = compare_chatglm_results("天水讼卦", base_model, qlora_model_e50_handmade, training_tag)

问题：天水讼卦

原始输出：
天水讼卦是周易中的一卦，代表诉诸枯骨，象征天无雨露，卦象为上卦坎（水），下卦乾（天），体卦为天，源卦为坎，大象代表天，象义为云雨聚于天际，示人雨后勤劳耕作的意义。

在周易中，讼卦被解释为加热的云雨，天无雨露时，需要人们lect（麦）耕种，分辨是非。这卦卦象表示，在雨后天晴时，要善于分辨积水和雨水从哪里流过，从而取得成功。因此，讼卦象征着人们追求正义时，需要坚持正见，不宜争执，避免诉讼，因为刚中有柔，能够成功。

讼卦的哲学内涵是：人们需要发挥自己的聪明才智，在取得成功前要善于等待时机，区分情况，不宜争执，不宜诉讼，应该谦逊等待他人认可后再行动。在实际应用中，讼卦提醒人们应该注意解决矛盾和问题，避免争执和诉讼，从而实现和平和谐。


微调后（ChatGLM3-6B(Epoch=50, handmade-dataset)）：
[gMASK]sop 天水讼卦, 卦象为:上卦为乾,下卦为坎, 中间卦为离。 卦象分析: 
上面的卦是乾，代表天，象征刚健坚强，但也有刚强过甚的特点。 

下面的卦是坎，代表水，象征灵活柔韧，但也有柔弱之疾的特点。 

中等的卦是离，代表火，象征热情，但也有过激情绪的特点。 

结合卦象来看，虽然天刚健坚强，但是水很灵活柔韧，过激情绪存在，所以需要火来中间平复。整体来看，需要刚健中庸，但避免过火。

  天水一极，地水二极。这是说卦象中，天在上而水在下，刚健过甚，需要火来中间平复。

  <卦象>
  <img src="http://www.qiankun.com/images/2017/09/tjqx_01_01_01_01.png" alt="<img src="http://www.qiankun.com/images/2017/09/tjqx_01_01_01_01.png" width="300" height="300" />\n\n<img src="http://www.qiankun.com/images/2017/09/tjqx_01_01_01_01.png" alt="<img src="http://www.qiankun.com/images/2017/09/tjqx_01_01_01_01.png" width="300" height="300" />\n\n<img src="ht